# WSI - ćwiczenie 1.
### Zagadnienie przeszukiwania i podstawowe podejścia do niego


1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [195]:
import numpy as np
from plotly import graph_objs as go
import math
import pandas as pd

RNG = np.random.default_rng()

#### Definicje funkcji oraz ich pochodnych cząstkowych (gradientów)

In [196]:
def f(vect):
    assert vect.shape == (1,)
    return 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]
    
def g(vect):
    assert vect.shape == (2,)
    return 10*vect[1]**4 + 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]

def grad_f(vect):
    assert vect.shape == (1,)
    return np.array(40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10)

def grad_g(vect):
    assert vect.shape == (2,)
    return np.array([40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10,
                     40*vect[1]**3])

### 

#### Algorytm najszybszego spadku


In [197]:
def gradient_descent(start_point, beta, grad, stop_treshlod, num_iters):
    steps = np.array([start_point])
    point = start_point
    for _ in range(num_iters):
        theta = grad(point)
        if abs(theta.sum()) < stop_treshlod:
            break
        # check for overflow errors 
        # (any inf or Nan values in the output array)
        if abs(theta.sum()) > 100000:
            return steps, False
        point = point - beta * theta
        steps = np.append(steps, [point], 0)
    return steps, True


#### Generowanie wykresów


Wykres dla funkcji jednej zmiennej f(x):

In [198]:
max_r = 3
X = np.linspace(-max_r, max_r, 100)
Y = np.array([f(np.array([x])) for x in X])

pt = RNG.uniform(-max_r, max_r, 1)
dsc = gradient_descent(pt, 0.005, grad_f, 0.01, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([x for x in dsc[0]
                    if abs(x[0]) < max_r])
    XS = steps[:, 0]
    YS = np.array([f(x) for x in steps])

    layout = go.Layout(width=700, height=500,
                    title_text='Gradient Descent of single variable function',
                    plot_bgcolor='DarkSeaGreen')
    fig = go.Figure(data=[go.Scatter(x=X, y=Y, line=dict(color='DarkSlateGrey', width=3))], 
                    layout=layout)
    fig.add_trace(go.Scatter(x=XS, y=YS, mode='markers', 
                            marker=dict(size=6, color=YS,               
                            colorscale='Agsunset')))
    fig.show()

Wykres dla funkcji dwóch zmiennych g(x, y):

In [199]:
max_r = 3
l = np.linspace(-max_r, max_r, 100)
X, Y = l, l
Z = np.array([[g(np.array([x, y])) for x in X] for y in Y])

pt = RNG.uniform(-max_r, max_r, 2)
dsc = gradient_descent(pt, 0.005, grad_g, 0.1, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([pt for pt in dsc[0] if abs(pt[0]) < max_r and abs(pt[1]) < max_r])
    XS, YS = steps[:,0], steps[:,1]
    ZS = np.array([g(np.array([x, y])) for x, y in zip(XS, YS)])

    layout = go.Layout(width = 700, height =700,
                                title_text='Gradient Descent of double variable function')
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Emrld',
                                    opacity=0.5)], layout=layout)
    fig.add_scatter3d(x=XS, y=YS, z=ZS, mode='markers', 
                    marker=dict(size=4, color=ZS,               
                                colorscale='Agsunset'))
    fig.show()

#### Analiza wydajności funkcji najszybszego spadku w zależności od wartości współczynnika kroku

Z przeprowadzonych obserwacji wywnioskowałem, że potrzebuję co najmniej kilkuset iteracji (uruchomień dla punktów początkowych) by zbadać generalne zachowanie algorytmu. Badania przeprowadzałem głównie na obszarze od -3 do 3, na którym 100 kroków algorytmu było zdecydowanie wystarczające żeby funkcja zatrzymała się po osiągnięciu dolnego limitu wartości gradientu, oznaczonego współczynnikiem odcięcia.


In [200]:
vals = [g(p) for p in dsc[0]]
layout = go.Layout(width=700, height=500,
                title_text='g(x,y) function value for algorithm steps',
                plot_bgcolor='DarkSeaGreen')
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(mode='markers', marker_color='DarkSlateGrey', 
                         x=list(range(len(vals))), y=vals))
fig.show()


Współczynnik odcięcia (stop_treshold) ustawiłem na 0.01, co wydało mi się wystarczające żeby stwierdzić że funkcja wystarczajaco zbliżyła się do minimum lokalnego i się stamtąd nie ruszy.

Badanie jakości współczynników  beta
1. generowanie zakresu bety
2. dla kazdej bety uruchomic algorytm dla n losowych punktów dla i iteracji,
   dla kazdego uruchomienia zachowac wartosc funkcji dla ostatniego punktu w 
   liscie kroków zwroconej przez algorytm
3. spozadzic tabelki wartosci
4. narysowac wykresy (box plot, histogram) 

In [201]:
def test_beta(func, grad, beta: int, max_r, num_iters: int):
    end_points = []
    for _ in range(num_iters):
        p = RNG.uniform(max_r[0], max_r[1], 2)
        dsc = gradient_descent(p, beta, grad, 0.01, 100)
        if not dsc[1]:
            end_points.append(np.nan)
        else:
            end_points.append(func(dsc[0][-1]))
    return np.array(end_points)

In [202]:
b_arr = np.arange(0.0005, 0.02, 0.0005)
num_iters = 100
performance = {np.round(b, 4):(test_beta(g, grad_g, b, [-3, 3], num_iters)) for b in b_arr}

df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]

In [203]:
df

,0.0005,0.0010,0.0015,0.0020,0.0025,0.0030,0.0035,0.0040,0.0045,0.0050,...,0.0150,0.0155,0.0160,0.0165,0.0170,0.0175,0.0180,0.0185,0.0190,0.0195
0,-42.092059,-6.865884,-6.943804,-7.006223,-42.605383,-42.620251,-42.616739,-6.997511,-7.000301,-7.000663,...,-39.454827,-34.105869,NaN,-7.005793,NaN,NaN,NaN,NaN,NaN,NaN
1,-42.123042,-42.505094,-42.569638,-42.597195,-42.616851,-42.612795,-42.620072,-42.619041,-42.622137,-42.622386,...,NaN,NaN,-38.769133,NaN,-7.005853,NaN,-42.627068,-30.026351,-34.306822,-31.657452
2,-42.627676,-6.864861,-42.564912,-6.970983,-6.983765,-42.613835,-42.618499,-42.618842,-42.627679,-42.621912,...,NaN,NaN,NaN,-7.005802,NaN,-7.005865,NaN,NaN,NaN,-41.078900
3,-6.619926,-6.878016,-7.006318,-6.970976,-42.605623,-42.612044,-42.618782,-7.005258,-42.621985,-42.622098,...,NaN,-7.005735,NaN,-40.360969,-24.583489,NaN,-39.245202,NaN,-8.899294,-7.005952
4,-6.466521,-6.864778,-6.961075,-6.974669,-6.985024,-6.992325,-6.997934,-7.003465,-42.620797,-7.000799,...,NaN,NaN,NaN,-36.297593,NaN,-7.005871,NaN,-40.167218,-8.899623,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-6.622302,-42.488266,-6.979445,-6.971049,-6.983699,-42.615926,-6.995339,-42.620149,-42.620858,-42.622033,...,NaN,NaN,-38.767427,-7.005802,-7.005833,NaN,NaN,NaN,NaN,-5.863184
96,-42.171366,-6.880528,-42.575580,-6.972294,-7.004466,-42.626564,-42.625077,-42.619225,-6.999385,-7.000663,...,NaN,-39.076058,-7.005926,-7.005772,-7.005836,NaN,-35.706174,-12.713514,NaN,NaN
97,-42.098825,-6.866005,-42.565137,-6.971322,-42.605147,-42.618000,-6.996191,-6.997487,-42.622099,-7.000675,...,NaN,-39.076053,NaN,-37.257567,NaN,-41.387345,NaN,-7.005910,-7.005939,NaN
98,-6.460859,-42.486671,-6.945760,-42.593147,-42.605168,-6.990676,-42.618813,-7.006308,-42.621864,-7.001422,...,-35.674481,NaN,NaN,-7.005865,NaN,NaN,-7.005916,NaN,NaN,NaN


In [204]:
df.describe()

,0.0005,0.0010,0.0015,0.0020,0.0025,0.0030,0.0035,0.0040,0.0045,0.0050,...,0.0150,0.0155,0.0160,0.0165,0.0170,0.0175,0.0180,0.0185,0.0190,0.0195
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,40.000000,47.000000,39.000000,45.000000,43.000000,35.000000,39.000000,36.000000,38.000000,26.000000
mean,-25.822974,-24.941633,-25.475318,-24.431816,-27.645796,-27.297461,-27.657701,-28.372535,-30.152542,-28.018280,...,-21.663066,-25.736322,-22.565687,-21.639813,-23.137863,-24.342844,-22.434717,-15.612176,-19.865364,-24.893808
std,17.830772,17.817302,17.886293,17.897007,17.669166,17.724355,17.669637,17.538845,17.078171,17.607945,...,15.659998,15.033648,14.026883,15.323979,14.667026,14.780837,15.353407,12.120837,12.566664,14.419562
min,-42.627676,-42.627668,-42.627678,-42.627676,-42.627665,-42.627669,-42.627678,-42.627679,-42.627679,-42.627679,...,-39.455349,-39.076641,-38.829498,-40.361339,-41.484838,-42.029948,-42.627068,-42.625455,-34.307204,-42.624302
25%,-42.124330,-42.489464,-42.566328,-42.593171,-42.606759,-42.613114,-42.617387,-42.619699,-42.621308,-42.622172,...,-39.454820,-39.076057,-32.718710,-36.297172,-37.169642,-38.623094,-37.058169,-24.652972,-31.990651,-36.416955
50%,-41.305240,-36.761626,-42.564786,-7.006194,-42.605158,-42.611963,-42.616145,-42.618842,-42.620700,-42.621996,...,-7.006213,-34.105578,-32.530064,-7.006321,-25.111387,-25.919465,-19.563235,-7.005956,-8.899620,-30.748332
75%,-6.514049,-6.872814,-6.944330,-6.971676,-6.984669,-6.991572,-6.996522,-6.998626,-7.000648,-7.001411,...,-7.005711,-7.005858,-7.005805,-7.005805,-7.005860,-7.005873,-7.005895,-7.005915,-8.899293,-7.006133
max,0.812583,-6.863256,-6.943409,-6.970934,-6.983673,-6.990595,-6.994767,-6.997476,-6.829244,-7.000566,...,-7.005693,-7.005534,-7.003903,-7.005772,-6.822740,-7.004935,-7.005252,-6.993256,-7.005365,-5.863184


In [205]:
layout = go.Layout(width=700, height=500,
                title_text='Algorithm outputs in function of beta value',
                plot_bgcolor='DarkSeaGreen')
fig = go.Figure(layout=layout)
for _, row in df.iterrows():
    fig.add_trace(go.Scatter(mode='markers', marker_color='DarkSlateGrey', 
                             opacity=0.5, x=df.columns, y=row, showlegend = False))
fig.show()


In [206]:
fail_rate = df.isna().sum()/num_iters

layout = go.Layout(width=700, height=500,
                title_text='Beta fail (out of bounds) rate',
                plot_bgcolor='DarkSeaGreen')
fig = go.Figure(data=[go.Bar(x=df.columns, y=fail_rate, marker_color='DarkSlateGrey')], layout=layout)
fig.show()



In [207]:
best_b = df.columns[df.quantile(0.3).argmin()]
df[best_b].isna().sum()
print(best_b)

0.012


In [208]:
layout = go.Layout(width=700, height=500,
                title_text='Best beta outputs',
                plot_bgcolor='DarkSeaGreen')
fig = go.Figure(data=[go.Bar(y=df[best_b], marker_color='DarkSlateGrey')], layout=layout)
fig.update_yaxes(autorange="reversed")
fig.show()